In [44]:
import numpy as np
import pandas as pd
import re
import json

from IPython.display import display
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm


# For cralwer
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import xml.etree.ElementTree as et
import requests

import xml.etree.ElementTree as et
import xmltodict

# For visualization
from geopy.geocoders import Nominatim
import folium

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17328\943676461.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [3]:
def query_df_out(query_df,query,version=None):
    a = query_df[query_df["구분"]==query]
    if version =="all":
        b = a
    elif version == 1:
        b = a[query_df["항목구분"]==0]
    else:
        b = a[query_df["항목구분"]==1]
    return b

In [4]:
df_in = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="all")
df_out = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="out")
df_item = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="items")

In [6]:
df_in2 = df_in[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)
df_out2 = df_out[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)

In [7]:
cate_ = {}
for row,col in df_in2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

for row,col in df_out2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

In [8]:
url_main = "http://apis.data.go.kr/B460014/foodBankInfoService2"

In [9]:
encode = "%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D"
decode = "/xc7f1IdclYG5tj6DIg1cU51nehX/zyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA=="

In [10]:
query_1 = {
    "getUserInfo": "이용자", "getFcltyGrpInfo" : "이용시설단체", "getCntrbtrInfo":"기부자",
    "getRceptStat": "접수현황", "getProvdStat": "제공현황",
    "getCnttgInfo":"물품현황","getSpctrInfo":"센터통계", "getPreferInfo":"선호물품"
}

In [11]:
query_1_2 = {
    value:key for key,value in query_1.items()
}

# 센터통계 monthly data extraction

In [18]:
url_test = url_main + "/" + "getSpctrInfo" + "?serviceKey=" + encode
paras_ = {
        "stdrYm": 0,
    "numOfRows": 0,
    "pageNo": 0

}
mon = ["01","02","03","04","05","06","07","08","09","10","11","12"]
year = ["2017","2018","2019","2020","2021","2022","2023"]
out_list = []
for y in year:
    for m in mon:
        if y =="2023":
            if m in ["07","08","09","10","11","12"]:
                pass
            else:
                out_1 = int(y + m)
        else:
            out_1 = int(y+m)
        out_list.append(out_1)

In [26]:
url_test = url_main + "/" + "getSpctrInfo" + "?serviceKey=" + encode
paras_["stdrYm"] = "201901"
paras_["numOfRows"] = 1000
paras_["pageNo"] = 1
for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)
html = requests.get(url_test,timeout=10).text

In [36]:
df_all = pd.DataFrame()
error_list = []
tq = tqdm(out_list)
for date_ in tq:
    url_test = url_main + "/" + "getSpctrInfo" + "?serviceKey=" + encode
    print(date_)
    paras_["stdrYm"] = date_
    paras_["numOfRows"] = 1000
    paras_["pageNo"] = 1

    for key,value in paras_.items():
        url_test += "&" + key + "=" + str(value)
    try:
        html = requests.get(url_test,timeout=10).text
        result = xmltodict.parse(html)
        #print(result)
        dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
        out = pd.DataFrame(dictionay)
        for col_ in out.columns:
            target_ = df_item[df_item["구분"]==col_]
            dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
            col_list = []
            if len(target_)!=0:
                try:
                    data_ = out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
                except:
                    data_ = out[col_]
                try:
                    data_2 = [dic_tar[i] for i in data_.values]
                except:
                    pass
                out[col_] = data_2
            else:
                pass
        out = out.rename(columns={col:cate_[col] for col in out.columns}).fillna("")
        col_tar =  ["기준년월","지역코드","지원센터단위코드","지원센터구분코드","지원센터코드","지원센터주소","지원센터상세주소"]
        out = out[col_tar]
        df_all = pd.concat([df_all,out])
    except:
        error_list.append(date_)

  0%|          | 0/84 [00:00<?, ?it/s]

201701
201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904
201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202301
202302
202303
202304
202305
202306
202306
202306
202306
202306
202306
202306


In [37]:
error_list

[201801,
 201802,
 201803,
 201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 202208,
 202301,
 202302,
 202303,
 202304,
 202305,
 202306,
 202306,
 202306,
 202306,
 202306,
 202306,
 202306]

In [38]:
df_all = df_all.reset_index(drop=True)

### refine dataframe

In [39]:
df_all["지원센터주소"] = [i.split("(")[0] for i in df_all["지원센터주소"]]
df_all["지원센터주소"] = [i.replace("번길","").replace("번지","").strip() for i in df_all["지원센터주소"]]
df_all["지원센터상세주소"] = [i.replace("번길","").replace("번지","").strip() for i in df_all["지원센터상세주소"]]
df_all["주소"]=df_all["지원센터주소"] + " " + df_all["지원센터상세주소"]
reg = re.compile("(.+[0-9]+) \w+")
test_list = []
for i in range(len(df_all)):
    can_ = df_all.loc[i]["지원센터주소"]
    #display(can_)
    test_ = reg.findall(can_)
    if len(test_)==0:
        test_list.append(can_)
    else:
        test_list.extend(test_)

df_all["지원센터주소2"] = test_list
reg = re.compile("(.+[가-힣]+)([0-9]+.+)")
test_list = []
for i in range(len(df_all)):
    can_ = df_all.loc[i]["지원센터주소"]
    test_ = reg.findall(can_)
    if len(test_)==0:
        test_list.append(can_)
    else:
        b_ = test_[0][0] + " " + test_[0][1]
        test_list.append(b_)
df_all["지원센터주소3"] = test_list

In [40]:
cen_list_2 = list(df_all["지원센터주소3"].value_counts().keys())
link_ = []
for cen_ in cen_list_2:
    tar_ = df_all[df_all["지원센터주소3"]==cen_].iloc[0]    
    link_.append([cen_,tar_["지원센터단위코드"],tar_["지원센터코드"],tar_["지역코드"]])

In [52]:
df_all.to_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\getSpctrInfo.xlsx")

# visualization

In [41]:
region_list = list(df_all["지역코드"].value_counts().keys())

In [42]:
out = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\lat_lng.xlsx")
out.head()

,Unnamed: 0,지원센터주소3,지원센터단위코드,지원센터코드,지역코드,refined_주소,lat,lng
0,0,경기도 수원시 팔달구 매산로 86,기초,수원시푸드뱅크,경기,경기도 수원시 팔달구 매산로 86,37.266166,127.000913
1,1,인천광역시 강화군 강화읍 향나무길 22 12,기초,강화군푸드뱅크,인천,인천광역시 강화군 강화읍 향나무길 22,37.746208,126.479814
2,2,경기도 파주시 교하로 914,기초,파주시희망나눔푸드뱅크,경기,경기도 파주시 교하로 914,37.723178,126.735678
3,3,경상북도 구미시 문장로 110,기초,구미시푸드뱅크,경북,경상북도 구미시 문장로 110,36.135076,128.332424
4,4,전라남도 무안군 삼향읍 오룡 3길 22,광역,전남광역푸드뱅크,전남,전라남도 무안군 삼향읍 오룡3길 22,0.000000,0.000000


In [45]:
lat_,lng_ = out.loc[0]["lat"],out.loc[0]["lng"]
g_map = folium.Map(location=[lat_,lng_],zoom_start=18,tiles="Stamen Toner")
#icon = folium.features.CustomIcon('PASTE_URL', icon_size=(24, 24))

In [46]:
marker = folium.Marker(location=[lat_, lng_],popup="A",icon= folium.Icon(color="green",icon="star"))
marker.add_to(g_map)

In [47]:
col_list = ["red", "blue", "green", "purple", "orange", "darkred",
            "lightred", "beige", "darkblue", "darkgreen", "cadetblue",
            "darkpurple", "white", "pink", "lightblue", "lightgreen", "gray", "black", "lightgray"]

In [48]:
i = 0
for region in region_list:
    tar_ = region
    tar_df = out[out["지역코드"]==tar_]
    tar_df = tar_df[["지원센터단위코드","지원센터코드","lat","lng"]]
    tar_df["level_col"] = col_list[i]
    i += 1
    tar_df["shape"] = tar_df["지원센터단위코드"].map(lambda x: "home" if x=="기초" else ("flag"))
    for row,col in tar_df.iterrows():
        tar_col = col["level_col"]
        tar_pop = col["지원센터코드"]
        tar_shape = col["shape"]
        check_ = col["지원센터단위코드"]
        if check_ == "기초":
            marker = folium.Marker(location=[col["lat"], col["lng"]],popup=tar_pop,
                                   icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)
        else:
            marker = folium.Marker(location=[col["lat"], col["lng"]],popup=tar_pop,
                       icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)
            marker = folium.CircleMarker(location=[col["lat"], col["lng"]],popup=tar_pop
                                         ,radius=20
                                         ,icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)


In [50]:
g_map